In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import _init_paths
import os
import sys
import numpy as np
import argparse
import pprint
import pdb
import time
import cv2
import torch
import pickle
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim

from torch.utils.data.sampler import Sampler
import torchvision.transforms as transforms
import torchvision.datasets as dset
from scipy.misc import imread
from roi_data_layer.roidb import combined_roidb
from roi_data_layer.roibatchLoader import roibatchLoader
from model.utils.config import cfg, cfg_from_file, cfg_from_list, get_output_dir
from model.rpn.bbox_transform import clip_boxes
from model.nms.nms_wrapper import nms
from model.rpn.bbox_transform import bbox_transform_inv
from model.utils.net_utils import save_net, load_net, vis_detections
from model.utils.blob import im_list_to_blob
from model.faster_rcnn.vgg16 import vgg16
from model.faster_rcnn.resnet import resnet
from model.utils.net_utils import weights_normal_init, save_net, load_net, \
      adjust_learning_rate, save_checkpoint, clip_gradient
    

In [3]:
def _get_image_blob(im):
    """Converts an image into a network input.
    Arguments:
    im (ndarray): a color image in BGR order
    Returns:
    blob (ndarray): a data blob holding an image pyramid
    im_scale_factors (list): list of image scales (relative to im) used
      in the image pyramid
    """
    im_orig = im.astype(np.float32, copy=True)
    im_orig -= cfg.PIXEL_MEANS

    im_shape = im_orig.shape
    im_size_min = np.min(im_shape[0:2])
    im_size_max = np.max(im_shape[0:2])

    processed_ims = []
    im_scale_factors = []

    for target_size in cfg.TEST.SCALES:
        im_scale = float(target_size) / float(im_size_min)
        # Prevent the biggest axis from being more than MAX_SIZE
        if np.round(im_scale * im_size_max) > cfg.TEST.MAX_SIZE:
            im_scale = float(cfg.TEST.MAX_SIZE) / float(im_size_max)
        im = cv2.resize(im_orig, None, None, fx=im_scale, fy=im_scale,interpolation=cv2.INTER_LINEAR)
        im_scale_factors.append(im_scale)
        processed_ims.append(im)

    # Create a blob to hold the input images
    blob = im_list_to_blob(processed_ims)

    return blob, np.array(im_scale_factors)

In [4]:
cfg_from_file("cfgs/vgg16.yml")
set_cfgs = ['ANCHOR_SCALES', '[4, 8, 16, 32]', 'ANCHOR_RATIOS', '[0.5,1,2]']
cfg_from_list(set_cfgs)
np.random.seed(cfg.RNG_SEED)
load_model_path = os.path.join("models/res101/holly/good_models/faster_rcnn_1_1_117000.pth")
holly_classes = np.asarray(['__background__', 'head'])
isCuda = True
cfg.CUDA = isCuda

In [6]:
def getDiff(old_tensor, new_tensor, seq, detail=False):
    total_difference = 0
    diff = 0
    original = 0  
    if "Conv2d" in str(old_tensor) and not hasattr(old_tensor, "__getitem__") :
        old_shape = old_tensor.weight.shape[0]
        new_shape = new_tensor.weight.shape[0]
        diff = old_shape - new_shape
        original += old_shape
        total_difference += diff
        if detail:
            print(str(e_old) + "Difference = {}".format(diff))
    elif "Sequential" in str(old_tensor) and hasattr(old_tensor, "__getitem__"):
        for j,b in enumerate(old_tensor):
            bn_old = vars(b)
            bn_new = vars(new_tensor[j])
            convs_old = bn_old["_modules"]
            convs_new = bn_new["_modules"]
            for k,v in convs_old.items():
                if "conv" in k:
                    old_shape = v.weight.shape[0]
                    original += old_shape
                    new_shape = convs_new[k].weight.shape[0]
                    diff = old_shape - new_shape
                    if detail:
                        print("S {}/ B {}/ {} Difference = {}".format(seq, j, k, diff))
                    total_difference += diff
    return original, total_difference

In [7]:
def make_cuda(isCuda, tensor):
    if isCuda:
        return tensor.cuda()
    return tensor

In [8]:
def getNumpy(tensor):
    return tensor.data.cpu().numpy()

def sortFilters(filters):
    #Sort filters by L1 Norm
    c = filters.reshape(-1, filters.shape[0])
    c = np.linalg.norm(c, ord=1, axis=0)
    d = np.zeros([2, c.shape[0]])
    i = np.argsort(c)
    d[0,:] = i
    d[1,:] = c[i]
    return d
def removeFilters(filters, percent, threshold=None):
    #Remove filters from numpy 
    t = threshold
    if threshold is None:
        t = np.median(filters[1,:]) - np.std(filters[1,:])
    mask = filters[1, :] > t
    numRemoved_1 = mask.sum()
    numRemoved_2 = int(numRemoved_1 * ((100. - percent) / 100.))
    numOfZeros = numRemoved_1 - numRemoved_2
    newFilters = filters[:,numRemoved_2:].copy()
    newFilters[1, 0:numOfZeros] =0
    return newFilters

def removePercent(filters, KeepPercent, PrunePercent):
    #Remove filters from numpy 
    numOfRemove = filters.shape[1] * PrunePercent / 100.
    numRemoved_1 = int(numOfRemove)
    numRemoved_2 = int(numRemoved_1 * ((100. - KeepPercent) / 100.))
    numOfZeros = numRemoved_1 - numRemoved_2
    newFilters = filters[:,numRemoved_2:].copy()
    print("NumOfZeros:")
    print(numOfZeros)
    newFilters[1, 0:numOfZeros] =0
    return newFilters

def sortRecoverFilters(l1Array, filters):
    #Recover the order of the pruned filters
    b = np.argsort(l1Array[0, :])
    c = l1Array[:,b]
    c = c[0,:].astype(int)
    shape = np.asarray(filters.shape)
    shape[0] = c.shape[0]
    newFilters = np.zeros(shape)
    newFilters = filters[c,:,:,:]
    return newFilters
def sortRecoverBatch(bn_tensor, index):
    #Batch pruning 
    index.sort()
    bn_rmean = bn_tensor.running_mean.cpu().numpy()
    bn_tensor.running_mean = torch.from_numpy(bn_rmean[index]).float().cuda()
    bn_rvar = bn_tensor.running_var.cpu().numpy()
    bn_tensor.running_var = torch.from_numpy(bn_rvar[index]).float().cuda()
    bn_weight = bn_tensor.weight.data.cpu().numpy()
    bn_tensor.weight.data = torch.from_numpy(bn_weight[index]).float().cuda()
    bn_bias = bn_tensor.bias.data.cpu().numpy()
    bn_tensor.bias.data = torch.from_numpy(bn_bias[index]).float().cuda()

def pruneConvLayers(tensor, percent = 10, threshold = None):
    #Prune out channels of convolutional layers
    filters = getNumpy(tensor.weight)
    d = sortFilters(filters)
    e = removeFilters(d, percent, threshold)
    f = sortRecoverFilters(e, filters)
    tensor.weight.data = torch.from_numpy(f).float().cuda()
    tensor.out_channels = f.shape[0]
    rindexes = e[0,:].astype(int).copy()
    return tensor, rindexes

def pruneConvPercent(tensor, KeepPercent = 10, PrunePercent=10):
    #Prune out channels of convolutional layers
    filters = getNumpy(tensor.weight)
    d = sortFilters(filters)
    e = removePercent(d, KeepPercent, PrunePercent)
    f = sortRecoverFilters(e, filters)
    tensor.weight.data = torch.from_numpy(f).float().cuda()
    tensor.out_channels = f.shape[0]
    rindexes = e[0,:].astype(int).copy()
    return tensor, rindexes
def pruneConvLowest(tensor, numKeep):
    filters = getNumpy(tensor.weight)
    d = sortFilters(filters)
    e = d[:,0:numKeep]
    f = sortRecoverFilters(e, filters)
    tensor.weight.data = torch.from_numpy(f).float().cuda()
    tensor.out_channels = f.shape[0]
    rindexes = e[0,:].astype(int).copy()
    return tensor, rindexes
def pruneConvWithIndexes(tensor, rindexes):
    #Prune out channels of convolutional layers using indexes
    filters = getNumpy(tensor.weight)
    tensor.weight.data = torch.from_numpy(filters[rindexes]).float().cuda()
    tensor.out_channels = tensor.weight.data.shape[0]
    return tensor, rindexes
def pruneNextLayer(nextLayerTensor, prevOutput, rindexes=None):
    #Prune input channels of everything
    if "BatchNorm" in str(nextLayerTensor):
        nextLayerTensor.num_features = prevOutput
        sortRecoverBatch(nextLayerTensor, rindexes)
    elif "Conv2d" in str(nextLayerTensor):
        if nextLayerTensor.weight.shape[1] == prevOutput:
            return
        nextLayerTensor.in_channels = prevOutput
        nextConvWeight = getNumpy(nextLayerTensor.weight)
        if not rindexes is None:
            c = nextConvWeight[:,rindexes,:,:]
        else:
            c = nextConvWeight
        nextLayerTensor.weight.data = torch.from_numpy(c).float().cuda()
    elif "Linear" in str(nextLayerTensor):
        n = getNumpy(nextLayerTensor.weight)
        fc = n[:,rindexes]
        nextLayerTensor.in_features = fc.shape[1]
        nextLayerTensor.weight.data = torch.from_numpy(fc).float().cuda()
    return nextLayerTensor
def prune_bottleneck(bottlenecks, prevOutput, rindexesX, keeplast=False):
    for i,bottleneck in enumerate(bottlenecks):
        #Conv1 Prune
        currentTensor = pruneNextLayer(bottleneck.conv1, prevOutput, rindexesX) #Conv1 Input channels Prune
        currentTensor, rindexes = pruneConvLayers(bottleneck.conv1) #Conv1 Output channels Prune
        currentTensor           = pruneNextLayer(bottleneck.bn1, currentTensor.out_channels, rindexes) #BN1 prune
        
        #Conv2 Prune
        currentTensor           = pruneNextLayer(bottleneck.conv2, currentTensor.num_features, rindexes) #Conv2 Input channels Prune
        currentTensor, rindexes = pruneConvLayers(bottleneck.conv2) #Conv2 Input channels Prune
        currentTensor           = pruneNextLayer(bottleneck.bn2, currentTensor.out_channels, rindexes) #BN2 prune
        #Shortcut + conv2
        currentTensor           = pruneNextLayer(bottleneck.conv3, currentTensor.num_features, rindexes) #Conv3 input channels
        
        #Prune shortcut first
        if not bottleneck.downsample is None:
            b_neck = bottleneck.downsample
            pruneNextLayer(b_neck[0], prevOutput, rindexesX) #Shortcut Input Channels
            shorcutconv, rindexes = pruneConvLayers(b_neck[0]) #Shortcut output channels
            pruneNextLayer(b_neck[1], b_neck[0].out_channels, rindexes) #bn1 of shorcut
        
            if not keeplast:
                lastconv, rindexes = pruneConvWithIndexes(bottleneck.conv3, rindexes) #Conv3 output channels
            else:
                lastconv = bottleneck.conv3
        else:
            if not keeplast:
                lastconv, rindexes = pruneConvLowest(bottleneck.conv3, len(rindexesX))
            else:
                lastconv = bottleneck.conv3
            
        lastbn = pruneNextLayer(bottleneck.bn3, currentTensor.out_channels, rindexes) #BN3 prune
        num_output = lastconv.out_channels
        rindexesX = rindexes
    return bottleneck, num_output, rindexes

def prune_bottleneck_percent(bottlenecks, rindexesX, percents):
    for i,bottleneck in enumerate(bottlenecks):
        #Conv1 Prune
        currentTensor = pruneNextLayer(bottleneck.conv1, len(rindexesX), rindexesX) #Conv1 Input channels Prune
        currentTensor, rindexes = pruneConvPercent(bottleneck.conv1, percents[0]) #Conv1 Output channels Prune
        currentTensor           = pruneNextLayer(bottleneck.bn1, currentTensor.out_channels, rindexes) #BN1 prune
        
        #Conv2 Prune
        currentTensor           = pruneNextLayer(bottleneck.conv2, currentTensor.num_features, rindexes) #Conv2 Input channels Prune
        currentTensor, rindexes = pruneConvPercent(bottleneck.conv2, percents[1]) #Conv2 Input channels Prune
        currentTensor           = pruneNextLayer(bottleneck.bn2, currentTensor.out_channels, rindexes) #BN2 prune
        #Shortcut + conv2
        currentTensor           = pruneNextLayer(bottleneck.conv3, currentTensor.num_features, rindexes) #Conv3 input channels
        
        #Prune shortcut first
        if not bottleneck.downsample is None:
            b_neck = bottleneck.downsample
            pruneNextLayer(b_neck[0], len(rindexesX), rindexesX) #Shortcut Input Channels
            shorcutconv, rindexes = pruneConvPercent(b_neck[0], percents[2]) #Shortcut output channels
            pruneNextLayer(b_neck[1], b_neck[0].out_channels, rindexes) #bn1 of shorcut
        
            lastconv, rindexes = pruneConvWithIndexes(bottleneck.conv3, rindexes) #Conv3 output channels
        else:
            lastconv, rindexes = pruneConvLowest(bottleneck.conv3, len(rindexesX))
            
        lastbn = pruneNextLayer(bottleneck.bn3, currentTensor.out_channels, rindexes) #BN3 prune
        num_output = lastconv.out_channels
        rindexesX = rindexes
    return bottleneck, num_output, rindexes


In [9]:
def pruneEverything():
    percent = 10
    prevIndex = 0
    prevOutput = 3
    rindexes = None
    for i,e in enumerate(fasterRCNN.RCNN_base):
        currentTensor = e
        if "Conv2d" in str(currentTensor) and not hasattr(currentTensor, "__getitem__"):
            if prevOutput != currentTensor.in_channels:
                currentTensor = pruneNextLayer(currentTensor, prevOutput)
            currentTensor, rindexes = pruneConvLayers(currentTensor, percent)
            prevOutput = currentTensor.out_channels

        elif "BatchNorm" in str(currentTensor) and not hasattr(currentTensor, "__getitem__"):
            if prevOutput != currentTensor.num_features:
                currentTensor = pruneNextLayer(currentTensor, prevOutput, rindexes)           
            prevOutput = currentTensor.num_features
        elif hasattr(currentTensor, "__getitem__"):
            b, prevOutput, rindexes = prune_bottleneck(currentTensor, prevOutput, rindexes)

        prevIndex = i
        prevTensor = currentTensor

    #Prune RoI pooling and FC classifiers
    b = pruneNextLayer(fasterRCNN.RCNN_rpn.RPN_Conv, prevOutput, rindexes)
    b, p, r = prune_bottleneck(fasterRCNN.RCNN_top[0], prevOutput, rindexes)
    fc = pruneNextLayer(fasterRCNN.RCNN_bbox_pred, p, r)
    cls_score = pruneNextLayer(fasterRCNN.RCNN_cls_score, p, r)

In [10]:
load_model_path = "pth_train_dir/vgg16/finished_models/vgg16_test_2_60000.pth"
fasterRCNN = vgg16(holly_classes, pretrained=False, class_agnostic=False)
fasterRCNN.create_architecture()
checkpoint = torch.load(load_model_path)
fasterRCNN.load_state_dict(checkpoint['model'])
if 'pooling_mode' in checkpoint.keys():
    cfg.POOLING_MODE = checkpoint['pooling_mode']

In [11]:
def reload_model(weightsPTH):
    fasterRCNN = vgg16(holly_classes, pretrained=False, class_agnostic=False)
    fasterRCNN.create_architecture()
    checkpoint = torch.load(weightsPTH)
    fasterRCNN.load_state_dict(checkpoint['model'])
    fasterRCNN.cuda()
    if 'pooling_mode' in checkpoint.keys():
        cfg.POOLING_MODE = checkpoint['pooling_mode']
    return fasterRCNN

In [12]:
if isCuda:
    fasterRCNN.cuda()

In [21]:
def evalNetwork(modelFR, out=False):
    modelFR.eval()
    
    cfg.TRAIN.USE_FLIPPED = False
    imdb_name = "holly_test"
    imdb, roidb, ratio_list, ratio_index = combined_roidb(imdb_name, False)
    imdb.competition_mode(on=True)
    
    save_name = 'faster_rcnn_10'
    num_images = len(imdb.image_index)
    all_boxes = [[[] for _ in range(num_images)]
               for _ in range(imdb.num_classes)]

    output_dir = get_output_dir(imdb, save_name)

    dataset = roibatchLoader(roidb, ratio_list, ratio_index, 1, \
                            imdb.num_classes, training=False, normalize = False)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=1,
                                shuffle=False, num_workers=0,
                                pin_memory=True)
    output_dir = "/tmp/outTest"
    vis = False
    thresh = 0.0
    
    data_iter = iter(dataloader)
    num_images = len(imdb.image_index)

    start = time.time()
    max_per_image = 100

    im_dataC = torch.FloatTensor(1)
    im_infoC = torch.FloatTensor(1)
    num_boxesC = torch.LongTensor(1)
    gt_boxesC = torch.FloatTensor(1)

    # ship to cuda
    im_dataC = make_cuda(isCuda, im_dataC)
    im_infoC = make_cuda(isCuda, im_infoC)
    num_boxesC = make_cuda(isCuda, num_boxesC)
    gt_boxesC = make_cuda(isCuda, gt_boxesC)

      # make variable
    im_dataC = Variable(im_dataC, volatile=True)
    im_infoC = Variable(im_infoC, volatile=True)
    num_boxesC = Variable(num_boxesC, volatile=True)
    gt_boxesC = Variable(gt_boxesC, volatile=True)

    _t = {'im_detect': time.time(), 'misc': time.time()}
    det_file = os.path.join(output_dir, 'detections.pkl')

    empty_array = np.transpose(np.array([[],[],[],[],[]]), (1,0))
    for i in range(num_images):

        data = next(data_iter)
        im_dataC.data.resize_(data[0].size()).copy_(data[0])
        im_infoC.data.resize_(data[1].size()).copy_(data[1])
        gt_boxesC.data.resize_(data[2].size()).copy_(data[2])
        num_boxesC.data.resize_(data[3].size()).copy_(data[3])

        det_tic = time.time()
        rois, cls_prob, bbox_pred, \
        rpn_loss_cls, rpn_loss_box, \
        RCNN_loss_cls, RCNN_loss_bbox, \
        rois_label = modelFR(im_dataC, im_infoC, gt_boxesC, num_boxesC)

        scores = cls_prob.data
        boxes = rois.data[:, :, 1:5]

        if cfg.TEST.BBOX_REG:
              # Apply bounding-box regression deltas
            box_deltas = bbox_pred.data
            if cfg.TRAIN.BBOX_NORMALIZE_TARGETS_PRECOMPUTED:
              # Optionally normalize targets by a precomputed mean and stdev
                if False:#args.class_agnostic:
                    box_deltas = box_deltas.view(-1, 4) * torch.FloatTensor(cfg.TRAIN.BBOX_NORMALIZE_STDS).cuda() \
                               + torch.FloatTensor(cfg.TRAIN.BBOX_NORMALIZE_MEANS).cuda()
                    box_deltas = box_deltas.view(1, -1, 4)
                else:
                    box_deltas = box_deltas.view(-1, 4) * torch.FloatTensor(cfg.TRAIN.BBOX_NORMALIZE_STDS).cuda() \
                               + torch.FloatTensor(cfg.TRAIN.BBOX_NORMALIZE_MEANS).cuda()
                    box_deltas = box_deltas.view(1, -1, 4 * len(imdb.classes))

            pred_boxes = bbox_transform_inv(boxes, box_deltas, 1)
            pred_boxes = clip_boxes(pred_boxes, im_infoC.data, 1)
        else:
              # Simply repeat the boxes, once for each class
            pred_boxes = np.tile(boxes, (1, scores.shape[1]))

        pred_boxes /= data[1][0][2]

        scores = scores.squeeze()
        pred_boxes = pred_boxes.squeeze()
        det_toc = time.time()
        detect_time = det_toc - det_tic
        misc_tic = time.time()
        if vis:
            im = cv2.imread(imdb.image_path_at(i))
            im2show = np.copy(im)
        for j in range(1, imdb.num_classes):
            inds = torch.nonzero(scores[:,j]>thresh).view(-1)
              # if there is det
            if inds.numel() > 0:
                cls_scores = scores[:,j][inds]
                _, order = torch.sort(cls_scores, 0, True)
                if False:#args.class_agnostic:
                    cls_boxes = pred_boxes[inds, :]
                else:
                    cls_boxes = pred_boxes[inds][:, j * 4:(j + 1) * 4]

                cls_dets = torch.cat((cls_boxes, cls_scores.unsqueeze(1)), 1)
                # cls_dets = torch.cat((cls_boxes, cls_scores), 1)
                cls_dets = cls_dets[order]
                keep = nms(cls_dets, cfg.TEST.NMS)
                cls_dets = cls_dets[keep.view(-1).long()]
                if vis:
                    im2show = vis_detections(im2show, imdb.classes[j], cls_dets.cpu().numpy(), 0.3)
                all_boxes[j][i] = cls_dets.cpu().numpy()
            else:
                all_boxes[j][i] = empty_array

          # Limit to max_per_image detections *over all classes*
        if max_per_image > 0:
            image_scores = np.hstack([all_boxes[j][i][:, -1]
                                        for j in range(1, imdb.num_classes)])
            if len(image_scores) > max_per_image:
                image_thresh = np.sort(image_scores)[-max_per_image]
                for j in range(1, imdb.num_classes):
                    keep = np.where(all_boxes[j][i][:, -1] >= image_thresh)[0]
                    all_boxes[j][i] = all_boxes[j][i][keep, :]

        misc_toc = time.time()
        nms_time = misc_toc - misc_tic

        sys.stdout.write('im_detect: {:d}/{:d} {:.3f}s {:.3f}s   \r' \
          .format(i + 1, num_images, detect_time, nms_time))
        sys.stdout.flush()

        if vis:
            cv2.imwrite('result.png', im2show)
            pdb.set_trace()

    with open(det_file, 'wb+') as f:
        pickle.dump(all_boxes, f, pickle.HIGHEST_PROTOCOL)

    print('Evaluating detections')
    ap = imdb.evaluate_detections(all_boxes, output_dir, out)
    return ap

In [22]:
def pruneConv(convTensor, nextConv, percent, KeepPercent = 10):
    convT = convTensor
    o_channels = convT.out_channels
    print(o_channels)
    _, r = pruneConvPercent(convT, KeepPercent, percent)
    bias_data = convT.bias.data.cpu().numpy()
    convT.bias.data = torch.from_numpy(bias_data[r]).float().cuda()
    print("PruneFiltersNum : {}".format(o_channels - r.shape[0]))
    pruneNextLayer(nextConv, r.shape[0], r)
    return r

In [23]:
def pruneConvBias(convTensor, percent, KeepPercent = 10):
    convT = convTensor
    o_channels = convT.out_channels
    print(o_channels)
    _, r = pruneConvPercent(convT, KeepPercent, percent)
    bias_data = convT.bias.data.cpu().numpy()
    convT.bias.data = torch.from_numpy(bias_data[r]).float().cuda()
    print("PruneFiltersNum : {}".format(o_channels - r.shape[0]))
    return convT, r

In [24]:
prevOutput = 3
compressionRate = 50
KeepPercent = 0
for i,e in enumerate(fasterRCNN.RCNN_base):
    if i == 0:
        prevOutput = e.out_channels
        continue
    if i == 28:
        currentTensor = pruneNextLayer(e, prevOutput, rindexes)
        continue

    currentTensor = e
    if "Conv2d" in str(currentTensor) and not hasattr(currentTensor, "__getitem__"):
        if prevOutput != currentTensor.in_channels:
            currentTensor = pruneNextLayer(currentTensor, prevOutput, rindexes)
        currentTensor, rindexes = pruneConvBias(currentTensor, compressionRate, KeepPercent)
        prevOutput = currentTensor.out_channels
    prevIndex = i
    prevTensor = currentTensor
    


64
NumOfZeros:
0
PruneFiltersNum : 32
128
NumOfZeros:
0
PruneFiltersNum : 64
128
NumOfZeros:
0
PruneFiltersNum : 64
256
NumOfZeros:
0
PruneFiltersNum : 128
256
NumOfZeros:
0
PruneFiltersNum : 128
256
NumOfZeros:
0
PruneFiltersNum : 128
512
NumOfZeros:
0
PruneFiltersNum : 256
512
NumOfZeros:
0
PruneFiltersNum : 256
512
NumOfZeros:
0
PruneFiltersNum : 256
512
NumOfZeros:
0
PruneFiltersNum : 256
512
NumOfZeros:
0
PruneFiltersNum : 256


In [16]:
evalNetwork(fasterRCNN)

Loaded dataset `HF__test` for training
Set proposal method: gt
Preparing training data...
HF__test gt roidb loaded from /export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/data/cache/HF__test_gt_roidb.pkl
done


/export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/lib/model/rpn/rpn.py:68: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  rpn_cls_prob_reshape = F.softmax(rpn_cls_score_reshape)


/export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/lib/model/faster_rcnn/faster_rcnn.py:97: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  cls_prob = F.softmax(cls_score)


Evaluating detections0.088s 0.002s   
Writing head VOC results file


0.0018251287121372713

In [32]:
checkpoint = torch.load("pth_train_dir/vgg16/pruned_l1_50/vgg16/holly/faster_rcnn_1_1_70000.pth")
fasterRCNN.load_state_dict(checkpoint['model'])

In [33]:
evalNetwork(fasterRCNN)

Loaded dataset `HF__test` for training
Set proposal method: gt
Preparing training data...
HF__test gt roidb loaded from /export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/data/cache/HF__test_gt_roidb.pkl
done


/export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/lib/model/rpn/rpn.py:68: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  rpn_cls_prob_reshape = F.softmax(rpn_cls_score_reshape)


/export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/lib/model/faster_rcnn/faster_rcnn.py:97: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  cls_prob = F.softmax(cls_score)


Evaluating detections1.066s 0.001s   
Writing head VOC results file


0.6254151349125667

In [17]:
save_name_model = os.path.join("./pth_train_dir/vgg16", 'pruned_50_noKeep.p')
torch.save(fasterRCNN, save_name_model)

In [22]:
imdb_name = "holly_trainval"
imdb, roidb, ratio_list, ratio_index = combined_roidb(imdb_name)

Loaded dataset `HF__trainval` for training
Set proposal method: gt
Preparing training data...
HF__trainval gt roidb loaded from /export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/data/cache/HF__trainval_gt_roidb.pkl
done
before filtering, there are 221643 images...
after filtering, there are 221643 images...


In [23]:
class sampler(Sampler):
    def __init__(self, train_size, batch_size):
        self.num_data = train_size
        self.num_per_batch = int(train_size / batch_size)
        self.batch_size = batch_size
        self.range = torch.arange(0,batch_size).view(1, batch_size).long()
        self.leftover_flag = False
        if train_size % batch_size:
            self.leftover = torch.arange(self.num_per_batch*batch_size, train_size).long()
            self.leftover_flag = True

    def __iter__(self):
        rand_num = torch.randperm(self.num_per_batch).view(-1,1) * self.batch_size
        self.rand_num = rand_num.expand(self.num_per_batch, self.batch_size) + self.range
        self.rand_num_view = self.rand_num.view(-1)

        if self.leftover_flag:
            self.rand_num_view = torch.cat((self.rand_num_view, self.leftover),0)

        return iter(self.rand_num_view)

    def __len__(self):
        return self.num_data

In [19]:
def fineTuneModel(model, imdb):
    batch_size = 2
    cfg.TRAIN.USE_FLIPPED = True
    cfg.USE_GPU_NMS = True
    start_epoch = 0
    max_epochs = 1
    lr_decay_step = 5
    lr_decay_gamma = 0.1
    disp_interval = 100
    output_dir = "./pth_train_dir/vgg16/pruned"
    class_agnostic = False
    session = 1
    train_size = len(roidb)
    lr = 0.0001 #cfg.TRAIN.LEARNING_RATE
    
    sampler_batch = sampler(train_size, batch_size)
    dataset_train = roibatchLoader(roidb, ratio_list, ratio_index, batch_size, imdb.num_classes, training=True)
    dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, sampler=sampler_batch, num_workers=1)
    im_dataT = torch.FloatTensor(1)
    im_infoT = torch.FloatTensor(1)
    num_boxesT = torch.LongTensor(1)
    gt_boxesT = torch.FloatTensor(1)

    # ship to cuda
    im_dataT = make_cuda(isCuda, im_dataT)
    im_infoT = make_cuda(isCuda, im_infoT)
    num_boxesT = make_cuda(isCuda, num_boxesT)
    gt_boxesT = make_cuda(isCuda, gt_boxesT)

      # make variable
    im_dataT = Variable(im_dataT)
    im_infoT = Variable(im_infoT)
    num_boxesT = Variable(num_boxesT)
    gt_boxesT = Variable(gt_boxesT)

    
    params = []
    for key, value in dict(model.named_parameters()).items():
        if value.requires_grad:
            if 'bias' in key:
                params += [{'params':[value],'lr':lr*(cfg.TRAIN.DOUBLE_BIAS + 1), \
                            'weight_decay': cfg.TRAIN.BIAS_DECAY and cfg.TRAIN.WEIGHT_DECAY or 0}]
            else:
                params += [{'params':[value],'lr':lr, 'weight_decay': cfg.TRAIN.WEIGHT_DECAY}]

    optimizer = torch.optim.SGD(params, momentum=cfg.TRAIN.MOMENTUM)
    iters_per_epoch = int(train_size / batch_size)

    for epoch in range(start_epoch, max_epochs + 1):
        # setting to train mode
        model.train()
        loss_temp = 0
        start = time.time()

        if epoch % (lr_decay_step + 1) == 0:
            adjust_learning_rate(optimizer, lr_decay_gamma)
            lr *= lr_decay_gamma

        data_iter = iter(dataloader_train)
        for step in range(iters_per_epoch):
            data = next(data_iter)
            im_dataT.data.resize_(data[0].size()).copy_(data[0])
            im_infoT.data.resize_(data[1].size()).copy_(data[1])
            gt_boxesT.data.resize_(data[2].size()).copy_(data[2])
            num_boxesT.data.resize_(data[3].size()).copy_(data[3])

            model.zero_grad()
            rois, cls_prob, bbox_pred, \
            rpn_loss_cls, rpn_loss_box, \
            RCNN_loss_cls, RCNN_loss_bbox, \
            rois_label = model(im_dataT, im_infoT, gt_boxesT, num_boxesT)

            loss = rpn_loss_cls.mean() + rpn_loss_box.mean() \
               + RCNN_loss_cls.mean() + RCNN_loss_bbox.mean()
            loss_temp += loss.data[0]

            # backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if step % disp_interval == 0:
                save_name = os.path.join(output_dir, 'faster_rcnn_{}_{}_{}.pth'.format(session, epoch, step))
                save_checkpoint({
                    'session': session,
                    'epoch': epoch + 1,
                    'model': fasterRCNN.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'pooling_mode': cfg.POOLING_MODE,
                    'class_agnostic': class_agnostic,
                    }, save_name)

                end = time.time()
                if step > 0:
                    loss_temp /= disp_interval

                loss_rpn_cls = rpn_loss_cls.data[0]
                loss_rpn_box = rpn_loss_box.data[0]
                loss_rcnn_cls = RCNN_loss_cls.data[0]
                loss_rcnn_box = RCNN_loss_bbox.data[0]
                fg_cnt = torch.sum(rois_label.data.ne(0))
                bg_cnt = rois_label.data.numel() - fg_cnt

                print("[session %d][epoch %2d][iter %4d/%4d] loss: %.4f, lr: %.2e" \
                                        % (session, epoch, step, iters_per_epoch, loss_temp, lr))
                print("\t\t\tfg/bg=(%d/%d), time cost: %f" % (fg_cnt, bg_cnt, end-start))
                print("\t\t\trpn_cls: %.4f, rpn_box: %.4f, rcnn_cls: %.4f, rcnn_box %.4f" \
                              % (loss_rpn_cls, loss_rpn_box, loss_rcnn_cls, loss_rcnn_box))

                loss_temp = 0
                start = time.time()


In [57]:
fineTuneModel(fasterRCNN, imdb)

/export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/lib/model/rpn/rpn.py:68: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  rpn_cls_prob_reshape = F.softmax(rpn_cls_score_reshape)
/export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/lib/model/faster_rcnn/faster_rcnn.py:97: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  cls_prob = F.softmax(cls_score)


[session 1][epoch  0][iter    0/111321] loss: 0.1360, lr: 1.00e-05
			fg/bg=(2/510), time cost: 11.909812
			rpn_cls: 0.1007, rpn_box: 0.0087, rcnn_cls: 0.0250, rcnn_box 0.0017
[session 1][epoch  0][iter  100/111321] loss: 0.6070, lr: 1.00e-05
			fg/bg=(28/484), time cost: 39.386562
			rpn_cls: 0.1476, rpn_box: 0.0024, rcnn_cls: 0.2305, rcnn_box 0.1086
[session 1][epoch  0][iter  200/111321] loss: 0.5978, lr: 1.00e-05
			fg/bg=(35/477), time cost: 39.440681
			rpn_cls: 0.3023, rpn_box: 0.0319, rcnn_cls: 0.2447, rcnn_box 0.1636
[session 1][epoch  0][iter  300/111321] loss: 0.4848, lr: 1.00e-05
			fg/bg=(42/470), time cost: 41.738666
			rpn_cls: 0.1766, rpn_box: 0.0075, rcnn_cls: 0.2853, rcnn_box 0.1894
[session 1][epoch  0][iter  400/111321] loss: 0.4629, lr: 1.00e-05
			fg/bg=(2/510), time cost: 40.823143
			rpn_cls: 0.0810, rpn_box: 0.0047, rcnn_cls: 0.0558, rcnn_box 0.0004
[session 1][epoch  0][iter  500/111321] loss: 0.4787, lr: 1.00e-05
			fg/bg=(34/478), time cost: 42.277877
			rp

[session 1][epoch  0][iter 4700/111321] loss: 0.4575, lr: 1.00e-05
			fg/bg=(2/510), time cost: 44.527432
			rpn_cls: 0.0676, rpn_box: 0.0004, rcnn_cls: 0.0670, rcnn_box 0.0001
[session 1][epoch  0][iter 4800/111321] loss: 0.4176, lr: 1.00e-05
			fg/bg=(28/484), time cost: 44.265397
			rpn_cls: 0.0768, rpn_box: 0.0124, rcnn_cls: 0.2023, rcnn_box 0.0975
[session 1][epoch  0][iter 4900/111321] loss: 0.4949, lr: 1.00e-05
			fg/bg=(4/508), time cost: 42.689468
			rpn_cls: 0.0777, rpn_box: 0.0033, rcnn_cls: 0.0846, rcnn_box 0.0001
[session 1][epoch  0][iter 5000/111321] loss: 0.4539, lr: 1.00e-05
			fg/bg=(35/477), time cost: 43.150548
			rpn_cls: 0.1540, rpn_box: 0.0244, rcnn_cls: 0.2446, rcnn_box 0.1724
[session 1][epoch  0][iter 5100/111321] loss: 0.4818, lr: 1.00e-05
			fg/bg=(93/419), time cost: 42.776473
			rpn_cls: 0.1738, rpn_box: 0.0335, rcnn_cls: 0.5496, rcnn_box 0.3834
[session 1][epoch  0][iter 5200/111321] loss: 0.4101, lr: 1.00e-05
			fg/bg=(2/510), time cost: 41.275376
			rpn

[session 1][epoch  0][iter 9400/111321] loss: 0.4705, lr: 1.00e-05
			fg/bg=(13/499), time cost: 45.012073
			rpn_cls: 0.1660, rpn_box: 0.0108, rcnn_cls: 0.1176, rcnn_box 0.0410
[session 1][epoch  0][iter 9500/111321] loss: 0.4767, lr: 1.00e-05
			fg/bg=(24/488), time cost: 45.378484
			rpn_cls: 0.0913, rpn_box: 0.0049, rcnn_cls: 0.1827, rcnn_box 0.1013
[session 1][epoch  0][iter 9600/111321] loss: 0.4553, lr: 1.00e-05
			fg/bg=(21/491), time cost: 43.609734
			rpn_cls: 0.1215, rpn_box: 0.0085, rcnn_cls: 0.1643, rcnn_box 0.0650
[session 1][epoch  0][iter 9700/111321] loss: 0.5342, lr: 1.00e-05
			fg/bg=(2/510), time cost: 44.249212
			rpn_cls: 0.1074, rpn_box: 0.0072, rcnn_cls: 0.0671, rcnn_box 0.0001
[session 1][epoch  0][iter 9800/111321] loss: 0.4754, lr: 1.00e-05
			fg/bg=(18/494), time cost: 43.919339
			rpn_cls: 0.1276, rpn_box: 0.0057, rcnn_cls: 0.1493, rcnn_box 0.0520
[session 1][epoch  0][iter 9900/111321] loss: 0.4760, lr: 1.00e-05
			fg/bg=(2/510), time cost: 44.117944
			rp

[session 1][epoch  0][iter 14000/111321] loss: 0.4875, lr: 1.00e-05
			fg/bg=(2/510), time cost: 45.844291
			rpn_cls: 0.0584, rpn_box: 0.0007, rcnn_cls: 0.0681, rcnn_box 0.0000
[session 1][epoch  0][iter 14100/111321] loss: 0.4727, lr: 1.00e-05
			fg/bg=(35/477), time cost: 44.294423
			rpn_cls: 0.1521, rpn_box: 0.0234, rcnn_cls: 0.2206, rcnn_box 0.1396
[session 1][epoch  0][iter 14200/111321] loss: 0.4481, lr: 1.00e-05
			fg/bg=(63/449), time cost: 42.612197
			rpn_cls: 0.0895, rpn_box: 0.0147, rcnn_cls: 0.3686, rcnn_box 0.2877
[session 1][epoch  0][iter 14300/111321] loss: 0.4501, lr: 1.00e-05
			fg/bg=(2/510), time cost: 43.619228
			rpn_cls: 0.0717, rpn_box: 0.0029, rcnn_cls: 0.0599, rcnn_box 0.0001
[session 1][epoch  0][iter 14400/111321] loss: 0.4985, lr: 1.00e-05
			fg/bg=(12/500), time cost: 43.649414
			rpn_cls: 0.2668, rpn_box: 0.0165, rcnn_cls: 0.1203, rcnn_box 0.0202
[session 1][epoch  0][iter 14500/111321] loss: 0.4792, lr: 1.00e-05
			fg/bg=(19/493), time cost: 42.835020

...

[session 1][epoch  0][iter 41600/111321] loss: 0.4332, lr: 1.00e-05
			fg/bg=(69/443), time cost: 52.841186
			rpn_cls: 0.1266, rpn_box: 0.0056, rcnn_cls: 0.2590, rcnn_box 0.1882
[session 1][epoch  0][iter 41700/111321] loss: 0.4231, lr: 1.00e-05
			fg/bg=(95/417), time cost: 49.004349
			rpn_cls: 0.0766, rpn_box: 0.0557, rcnn_cls: 0.4194, rcnn_box 0.3054
[session 1][epoch  0][iter 41800/111321] loss: 0.4944, lr: 1.00e-05
			fg/bg=(37/475), time cost: 47.217960
			rpn_cls: 0.0952, rpn_box: 0.0090, rcnn_cls: 0.2223, rcnn_box 0.1668
[session 1][epoch  0][iter 41900/111321] loss: 0.4289, lr: 1.00e-05
			fg/bg=(27/485), time cost: 43.997671
			rpn_cls: 0.1612, rpn_box: 0.0456, rcnn_cls: 0.1643, rcnn_box 0.1079
[session 1][epoch  0][iter 42000/111321] loss: 0.4655, lr: 1.00e-05
			fg/bg=(62/450), time cost: 47.337700
			rpn_cls: 0.1561, rpn_box: 0.0209, rcnn_cls: 0.2324, rcnn_box 0.2150
[session 1][epoch  0][iter 42100/111321] loss: 0.4414, lr: 1.00e-05
			fg/bg=(38/474), time cost: 45.6102

[session 1][epoch  0][iter 46200/111321] loss: 0.3829, lr: 1.00e-05
			fg/bg=(91/421), time cost: 47.420237
			rpn_cls: 0.1954, rpn_box: 0.0562, rcnn_cls: 0.2376, rcnn_box 0.2805
[session 1][epoch  0][iter 46300/111321] loss: 0.4752, lr: 1.00e-05
			fg/bg=(45/467), time cost: 42.951339
			rpn_cls: 0.1585, rpn_box: 0.0148, rcnn_cls: 0.2747, rcnn_box 0.1444
[session 1][epoch  0][iter 46400/111321] loss: 0.4758, lr: 1.00e-05
			fg/bg=(42/470), time cost: 48.002445
			rpn_cls: 0.0958, rpn_box: 0.0044, rcnn_cls: 0.1055, rcnn_box 0.1271
[session 1][epoch  0][iter 46500/111321] loss: 0.4197, lr: 1.00e-05
			fg/bg=(34/478), time cost: 44.458951
			rpn_cls: 0.1183, rpn_box: 0.0033, rcnn_cls: 0.2579, rcnn_box 0.1262
[session 1][epoch  0][iter 46600/111321] loss: 0.4535, lr: 1.00e-05
			fg/bg=(29/483), time cost: 45.629855
			rpn_cls: 0.1747, rpn_box: 0.0023, rcnn_cls: 0.3274, rcnn_box 0.1984
[session 1][epoch  0][iter 46700/111321] loss: 0.4655, lr: 1.00e-05
			fg/bg=(46/466), time cost: 46.0866

Process Process-2:
Traceback (most recent call last):
  File "/export/livia/home/vision/lethanh/anaconda2/envs/py3ml/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/export/livia/home/vision/lethanh/anaconda2/envs/py3ml/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/export/livia/home/vision/lethanh/anaconda2/envs/py3ml/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 50, in _worker_loop
    r = index_queue.get()
  File "/export/livia/home/vision/lethanh/anaconda2/envs/py3ml/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/export/livia/home/vision/lethanh/anaconda2/envs/py3ml/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/export/livia/home/vision/lethanh/anaconda2/envs/py3ml/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_b

Traceback (most recent call last):
  File "/export/livia/home/vision/lethanh/anaconda2/envs/py3ml/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-57-681c42224077>", line 1, in <module>
    fineTuneModel(fasterRCNN, imdb)
  File "<ipython-input-19-81ed733ebca0>", line 60, in fineTuneModel
    data = next(data_iter)
  File "/export/livia/home/vision/lethanh/anaconda2/envs/py3ml/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 275, in __next__
    idx, batch = self._get_batch()
  File "/export/livia/home/vision/lethanh/anaconda2/envs/py3ml/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 254, in _get_batch
    return self.data_queue.get()
  File "/export/livia/home/vision/lethanh/anaconda2/envs/py3ml/lib/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
  File "/export/livia/home/vision/lethanh/anaco

KeyboardInterrupt: 

In [21]:
checkpoint = torch.load("pth_train_dir/vgg16/pruned/faster_rcnn_1_0_74000.pth")
fasterRCNN.load_state_dict(checkpoint['model'])

In [58]:
evalNetwork(fasterRCNN)

Loaded dataset `HF__test` for training
Set proposal method: gt
Preparing training data...
HF__test gt roidb loaded from /export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/data/cache/HF__test_gt_roidb.pkl
done


/export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/lib/model/rpn/rpn.py:68: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  rpn_cls_prob_reshape = F.softmax(rpn_cls_score_reshape)
/export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/lib/model/faster_rcnn/faster_rcnn.py:97: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  cls_prob = F.softmax(cls_score)


Evaluating detections0.084s 0.001s   
Writing head VOC results file


Exception ignored in: <bound method DataLoaderIter.__del__ of <torch.utils.data.dataloader.DataLoaderIter object at 0x7f3f8c3e0630>>
Traceback (most recent call last):
  File "/export/livia/home/vision/lethanh/anaconda2/envs/py3ml/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 333, in __del__
    self._shutdown_workers()
  File "/export/livia/home/vision/lethanh/anaconda2/envs/py3ml/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 319, in _shutdown_workers
    self.data_queue.get()
  File "/export/livia/home/vision/lethanh/anaconda2/envs/py3ml/lib/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
  File "/export/livia/home/vision/lethanh/anaconda2/envs/py3ml/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 70, in rebuild_storage_fd
    fd = df.detach()
  File "/export/livia/home/vision/lethanh/anaconda2/envs/py3ml/lib/python3.6/multiprocessing/resource_sharer.py", line 57, in detach
   

0.11536614958388539

In [81]:
_,no, r = prune_bottleneck_percent(fasterRCNN.RCNN_base[4], r, [0, 0, 0])

NumOfZeros:
0
NumOfZeros:
0
NumOfZeros:
0
NumOfZeros:
0
NumOfZeros:
0
NumOfZeros:
0
NumOfZeros:
0


In [82]:
pruneNextLayer(fasterRCNN.RCNN_base[5][0].conv1, len(r), r)
pruneNextLayer(fasterRCNN.RCNN_base[5][0].downsample[0], len(r), r)

Conv2d(231, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)

In [83]:
ap = evalNetwork(fasterRCNN)

Loaded dataset `HF__test` for training
Set proposal method: gt
Preparing training data...
HF__test gt roidb loaded from /export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/data/cache/HF__test_gt_roidb.pkl
done


/export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/lib/model/rpn/rpn.py:68: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  rpn_cls_prob_reshape = F.softmax(rpn_cls_score_reshape)
/export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/lib/model/faster_rcnn/faster_rcnn.py:97: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  cls_prob = F.softmax(cls_score)


Evaluating detections0.208s 0.001s   
Writing head VOC results file


In [84]:
ap

0.0013666499637356947

In [48]:
test = reload_model(save_name_model, weightsPTH)

In [53]:
test.RCNN_base[4][0].conv1.weight.shape

torch.Size([8, 58, 1, 1])

In [54]:
b_1_1_conv1 = test.RCNN_base[4][0].conv1
o_channels = b_1_1_conv2.out_channels
print(o_channels)
b_1_1_conv2 = test.RCNN_base[4][0].conv2
b_1_1_bn1 = test.RCNN_base[4][0].bn1
_, r = pruneConvPercent(b_1_1_conv1, KeepPercent, 70)
print("PruneFiltersNum : {}".format(o_channels - r.shape[0]))
pruneNextLayer(b_1_1_bn1, r.shape[0], r)
pruneNextLayer(b_1_1_conv2, r.shape[0], r)
ap = evalNetwork(test)

64
NumOfZeros:
1
PruneFiltersNum : 60
Loaded dataset `HF__test` for training
Set proposal method: gt
Preparing training data...
HF__test gt roidb loaded from /export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/data/cache/HF__test_gt_roidb.pkl
done


/export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/lib/model/rpn/rpn.py:68: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  rpn_cls_prob_reshape = F.softmax(rpn_cls_score_reshape)
/export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/lib/model/faster_rcnn/faster_rcnn.py:97: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  cls_prob = F.softmax(cls_score)


Evaluating detections0.155s 0.001s   
Writing head VOC results file


In [55]:
ap

0.777952685259206

In [33]:
def prune1stConv(fasterRCNN):
    PrunePercent = 12
    KeepPercent = 10
    conv1st = fasterRCNN.RCNN_base[0]
    bn1 = fasterRCNN.RCNN_base[1]
    b_1_1_conv1 = fasterRCNN.RCNN_base[4][0].conv1
    b_1_1_shortcut = fasterRCNN.RCNN_base[4][0].downsample[0]
    _ , r = pruneConvPercent(conv1st, KeepPercent, PrunePercent)
    pruneNextLayer(bn1, r.shape[0], r)
    p_1_1_c1 = pruneNextLayer(b_1_1_conv1, r.shape[0], r)
    p_1_1_shortcut = pruneNextLayer(b_1_1_shortcut, r.shape[0], r)

In [34]:
#Pruning of first conv1 of block 1, bottleneck 1

weightsPTH = "models/res101/holly/faster_rcnn_1_1_52000.pth"
prunePercent = range(98, 100, 1)
originalAcc = 0.758
KeepPercent = 10
for i in prunePercent:
    print("PrunePercent : {}".format(i))
    fasterRCNN = reload_model(save_name_model, weightsPTH)
    b_1_1_conv1 = fasterRCNN.RCNN_base[4][0].conv1
    o_channels = b_1_1_conv1.out_channels
    b_1_1_conv2 = fasterRCNN.RCNN_base[4][0].conv2
    b_1_1_bn1 = fasterRCNN.RCNN_base[4][0].bn1
    _, r = pruneConvPercent(b_1_1_conv1, KeepPercent, i)
    print("PruneFiltersNum : {}".format(o_channels - r.shape[0]))
    pruneNextLayer(b_1_1_bn1, r.shape[0], r)
    pruneNextLayer(b_1_1_conv2, r.shape[0], r)
    ap = evalNetwork(fasterRCNN)
    print("AP: {}".format(ap))
    if np.abs(originalAcc - ap) > 0.4:
        break


PrunePercent : 98
NumOfZeros:
1
PruneFiltersNum : 6
Loaded dataset `HF__test` for training
Set proposal method: gt
Preparing training data...
HF__test gt roidb loaded from /export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/data/cache/HF__test_gt_roidb.pkl


KeyboardInterrupt: 

In [ ]:
#Pruning of first conv2 of block 1, bottleneck 1
weightsPTH = "models/res101/holly/faster_rcnn_1_1_52000.pth"
save_name_model = os.path.join("./models/res101/holly/", 'pruned_117000_2.p')
prunePercent = range(10, 100, 2)
originalAcc = 0.758
KeepPercent = 10
for i in prunePercent:
    print("PrunePercent : {}".format(i))
    fasterRCNN = reload_model(save_name_model, weightsPTH)
    b_1_1_conv2 = fasterRCNN.RCNN_base[4][0].conv2
    o_channels = b_1_1_conv2.out_channels
    b_1_1_conv3 = fasterRCNN.RCNN_base[4][0].conv3
    b_1_1_bn2 = fasterRCNN.RCNN_base[4][0].bn2
    _, r = pruneConvPercent(b_1_1_conv2, KeepPercent, i)
    print("PruneFiltersNum : {}".format(o_channels - r.shape[0]))
    pruneNextLayer(b_1_1_bn2, r.shape[0], r)
    pruneNextLayer(b_1_1_conv3, r.shape[0], r)
    ap = evalNetwork(fasterRCNN)
    print("AP: {}".format(ap))
    if np.abs(originalAcc - ap) > 0.4:
        break

PrunePercent : 10
NumOfZeros:
1
PruneFiltersNum : 5
Loaded dataset `HF__test` for training
Set proposal method: gt
Preparing training data...
HF__test gt roidb loaded from /export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/data/cache/HF__test_gt_roidb.pkl
done


/export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/lib/model/rpn/rpn.py:68: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  rpn_cls_prob_reshape = F.softmax(rpn_cls_score_reshape)
/export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/lib/model/faster_rcnn/faster_rcnn.py:97: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  cls_prob = F.softmax(cls_score)


Evaluating detections0.086s 0.001s   
Writing head VOC results file
AP: 0.7220566953398896
PrunePercent : 12
NumOfZeros:
1
PruneFiltersNum : 6
Loaded dataset `HF__test` for training
Set proposal method: gt
Preparing training data...
HF__test gt roidb loaded from /export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/data/cache/HF__test_gt_roidb.pkl
done
Evaluating detections0.109s 0.001s   
Writing head VOC results file
AP: 0.6984362425509368
PrunePercent : 14
NumOfZeros:
1
PruneFiltersNum : 7
Loaded dataset `HF__test` for training
Set proposal method: gt
Preparing training data...
HF__test gt roidb loaded from /export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/data/cache/HF__test_gt_roidb.pkl
done
Evaluating detections0.115s 0.001s   
Writing head VOC results file
AP: 0.6984362425509368
PrunePercent : 16
NumOfZeros:
1
PruneFiltersNum : 9
Loaded dataset `HF__test` for training
Set proposal method: gt
Preparing training data...
HF__test gt roidb loaded from /export

done
Evaluating detections0.123s 0.001s   
Writing head VOC results file
AP: 0.502514688747505
PrunePercent : 58
NumOfZeros:
4
PruneFiltersNum : 33
Loaded dataset `HF__test` for training
Set proposal method: gt
Preparing training data...
HF__test gt roidb loaded from /export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/data/cache/HF__test_gt_roidb.pkl
done
Evaluating detections0.104s 0.001s   
Writing head VOC results file
AP: 0.49872285141737127
PrunePercent : 60
NumOfZeros:
4
PruneFiltersNum : 34
Loaded dataset `HF__test` for training
Set proposal method: gt
Preparing training data...
HF__test gt roidb loaded from /export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/data/cache/HF__test_gt_roidb.pkl
done
Evaluating detections0.130s 0.002s   
Writing head VOC results file
AP: 0.49872285141737127
PrunePercent : 62
NumOfZeros:
4
PruneFiltersNum : 35
Loaded dataset `HF__test` for training
Set proposal method: gt
Preparing training data...
HF__test gt roidb loaded fro

In [56]:
fasterRCNN.RCNN_base

Sequential(
  (0): Conv2d(3, 58, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(58, eps=1e-05, momentum=0.1, affine=True)
  (2): ReLU(inplace)
  (3): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), dilation=(1, 1), ceil_mode=True)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(58, 8, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True)
      (conv2): Conv2d(8, 9, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(9, eps=1e-05, momentum=0.1, affine=True)
      (conv3): Conv2d(9, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(58, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      )
    )
    (1): Bottleneck(
      (conv1):

In [57]:
#Prune 
weightsPTH = "models/res101/holly/faster_rcnn_1_1_52000.pth"
save_name_model = os.path.join("./models/res101/holly/", 'pruned_117000_2.p')
prunePercent = range(10, 100, 10)
originalAcc = 0.55
KeepPercent = 10
for i in prunePercent:
    print("PrunePercent : {}".format(i))
    fasterRCNN = reload_model(save_name_model, weightsPTH)
    b_1_1_shortcut = fasterRCNN.RCNN_base[4][0].downsample[0]
    o_channels = b_1_1_conv1.out_channels
    b_1_2_conv1 = fasterRCNN.RCNN_base[4][1].conv1
    b_1_1_bn1 = fasterRCNN.RCNN_base[4][0].downsample[1]
    _, r = pruneConvPercent(b_1_1_conv1, KeepPercent, i)
    print("PruneFiltersNum : {}".format(o_channels - r.shape[0]))
    pruneNextLayer(b_1_1_bn1, r.shape[0], r)
    pruneNextLayer(b_1_1_conv2, r.shape[0], r)
    ap = evalNetwork(fasterRCNN)
    print("AP: {}".format(ap))
    if np.abs(originalAcc - ap) > 0.4:
        break
    


PrunePercent : 10
NumOfZeros:
0
PruneFiltersNum : 0
Loaded dataset `HF__test` for training
Set proposal method: gt
Preparing training data...
HF__test gt roidb loaded from /export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/data/cache/HF__test_gt_roidb.pkl
done


RuntimeError: running_mean should contain 256 elements not 4

In [40]:
b_1_1_conv3

Conv2d(9, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)

In [39]:
fasterRCNN.RCNN_base[4][0].conv2

Conv2d(8, 9, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)

In [ ]:
load_model_path = "models/res101/holly/faster_rcnn_1_1_52000.pth"
save_name_model = os.path.join("./models/res101/holly/", 'pruned_117000_2.p')
fasterRCNN = reload_model(save_name_model, load_model_path)

In [15]:
def evalNetwork(modelFR, out=False):
    modelFR.eval()
    
    cfg.TRAIN.USE_FLIPPED = False
    imdb_name = "holly_test"
    imdb, roidb, ratio_list, ratio_index = combined_roidb(imdb_name, False)
    imdb.competition_mode(on=True)
    
    save_name = 'faster_rcnn_10'
    num_images = len(imdb.image_index)
    all_boxes = [[[] for _ in range(num_images)]
               for _ in range(imdb.num_classes)]

    output_dir = get_output_dir(imdb, save_name)

    dataset = roibatchLoader(roidb, ratio_list, ratio_index, 1, \
                            imdb.num_classes, training=False, normalize = False)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=1,
                                shuffle=False, num_workers=0,
                                pin_memory=True)
    output_dir = "/tmp/outTest"
    vis = False
    thresh = 0.0
    
    data_iter = iter(dataloader)
    num_images = len(imdb.image_index)

    start = time.time()
    max_per_image = 100

    im_dataC = torch.FloatTensor(1)
    im_infoC = torch.FloatTensor(1)
    num_boxesC = torch.LongTensor(1)
    gt_boxesC = torch.FloatTensor(1)

    # ship to cuda
    im_dataC = make_cuda(isCuda, im_dataC)
    im_infoC = make_cuda(isCuda, im_infoC)
    num_boxesC = make_cuda(isCuda, num_boxesC)
    gt_boxesC = make_cuda(isCuda, gt_boxesC)

      # make variable
    im_dataC = Variable(im_dataC, volatile=True)
    im_infoC = Variable(im_infoC, volatile=True)
    num_boxesC = Variable(num_boxesC, volatile=True)
    gt_boxesC = Variable(gt_boxesC, volatile=True)

    _t = {'im_detect': time.time(), 'misc': time.time()}
    det_file = os.path.join(output_dir, 'detections.pkl')

    empty_array = np.transpose(np.array([[],[],[],[],[]]), (1,0))
    for i in range(num_images):

        data = next(data_iter)
        im_dataC.data.resize_(data[0].size()).copy_(data[0])
        im_infoC.data.resize_(data[1].size()).copy_(data[1])
        gt_boxesC.data.resize_(data[2].size()).copy_(data[2])
        num_boxesC.data.resize_(data[3].size()).copy_(data[3])

        det_tic = time.time()
        rois, cls_prob, bbox_pred, \
        rpn_loss_cls, rpn_loss_box, \
        RCNN_loss_cls, RCNN_loss_bbox, \
        rois_label = modelFR(im_dataC, im_infoC, gt_boxesC, num_boxesC)

        scores = cls_prob.data
        boxes = rois.data[:, :, 1:5]

        if cfg.TEST.BBOX_REG:
              # Apply bounding-box regression deltas
            box_deltas = bbox_pred.data
            if cfg.TRAIN.BBOX_NORMALIZE_TARGETS_PRECOMPUTED:
              # Optionally normalize targets by a precomputed mean and stdev
                if False:#args.class_agnostic:
                    box_deltas = box_deltas.view(-1, 4) * torch.FloatTensor(cfg.TRAIN.BBOX_NORMALIZE_STDS).cuda() \
                               + torch.FloatTensor(cfg.TRAIN.BBOX_NORMALIZE_MEANS).cuda()
                    box_deltas = box_deltas.view(1, -1, 4)
                else:
                    box_deltas = box_deltas.view(-1, 4) * torch.FloatTensor(cfg.TRAIN.BBOX_NORMALIZE_STDS).cuda() \
                               + torch.FloatTensor(cfg.TRAIN.BBOX_NORMALIZE_MEANS).cuda()
                    box_deltas = box_deltas.view(1, -1, 4 * len(imdb.classes))

            pred_boxes = bbox_transform_inv(boxes, box_deltas, 1)
            pred_boxes = clip_boxes(pred_boxes, im_infoC.data, 1)
        else:
              # Simply repeat the boxes, once for each class
            pred_boxes = np.tile(boxes, (1, scores.shape[1]))

        pred_boxes /= data[1][0][2]

        scores = scores.squeeze()
        pred_boxes = pred_boxes.squeeze()
        det_toc = time.time()
        detect_time = det_toc - det_tic
        misc_tic = time.time()
        if vis:
            im = cv2.imread(imdb.image_path_at(i))
            im2show = np.copy(im)
        for j in range(1, imdb.num_classes):
            inds = torch.nonzero(scores[:,j]>thresh).view(-1)
              # if there is det
            if inds.numel() > 0:
                cls_scores = scores[:,j][inds]
                _, order = torch.sort(cls_scores, 0, True)
                if False:#args.class_agnostic:
                    cls_boxes = pred_boxes[inds, :]
                else:
                    cls_boxes = pred_boxes[inds][:, j * 4:(j + 1) * 4]

                cls_dets = torch.cat((cls_boxes, cls_scores.unsqueeze(1)), 1)
                # cls_dets = torch.cat((cls_boxes, cls_scores), 1)
                cls_dets = cls_dets[order]
                keep = nms(cls_dets, cfg.TEST.NMS)
                cls_dets = cls_dets[keep.view(-1).long()]
                if vis:
                    im2show = vis_detections(im2show, imdb.classes[j], cls_dets.cpu().numpy(), 0.3)
                all_boxes[j][i] = cls_dets.cpu().numpy()
            else:
                all_boxes[j][i] = empty_array

          # Limit to max_per_image detections *over all classes*
        if max_per_image > 0:
            image_scores = np.hstack([all_boxes[j][i][:, -1]
                                        for j in range(1, imdb.num_classes)])
            if len(image_scores) > max_per_image:
                image_thresh = np.sort(image_scores)[-max_per_image]
                for j in range(1, imdb.num_classes):
                    keep = np.where(all_boxes[j][i][:, -1] >= image_thresh)[0]
                    all_boxes[j][i] = all_boxes[j][i][keep, :]

        misc_toc = time.time()
        nms_time = misc_toc - misc_tic

        sys.stdout.write('im_detect: {:d}/{:d} {:.3f}s {:.3f}s   \r' \
          .format(i + 1, num_images, detect_time, nms_time))
        sys.stdout.flush()

        if vis:
            cv2.imwrite('result.png', im2show)
            pdb.set_trace()

    with open(det_file, 'wb+') as f:
        pickle.dump(all_boxes, f, pickle.HIGHEST_PROTOCOL)

    print('Evaluating detections')
    ap = imdb.evaluate_detections(all_boxes, output_dir, out)
    return ap

In [16]:
evalNetwork(fasterRCNN)

Loaded dataset `HF__test` for training
Set proposal method: gt
Preparing training data...
HF__test gt roidb loaded from /export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/data/cache/HF__test_gt_roidb.pkl
done


/export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/lib/model/rpn/rpn.py:68: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  rpn_cls_prob_reshape = F.softmax(rpn_cls_score_reshape)


/export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/lib/model/faster_rcnn/faster_rcnn.py:97: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  cls_prob = F.softmax(cls_score)


Evaluating detections0.093s 0.001s   
Writing head VOC results file


0.7901467630563271

In [26]:
save_name_model = os.path.join("./models/res101/holly/", 'pruned_117000_2.p')
torch.save(fasterRCNN, save_name_model)